# PHW#4 Chapter 8

## `Inderpreet Singh`

## \`Video link: sorry couldent make the video had a midterm and been studying for it late night adn the exam was for CS344

## `CSCI-381`

### **Chapter 8**
**Exercise: 6 - Create the tables `dbo.Orders` and `dbo.OrderDetails` as specified and populate them with initial data.**

---

#### **Tables Involved**:

1. **dbo.Orders**: Stores basic information about each order, including `OrderID`, `CustomerID`, `OrderDate`, `ShippedDate`, `Freight`, `ShipName`, and various shipping address details.
2. **dbo.OrderDetails**: Stores individual item details for each order, linked to the `dbo.Orders` table through `OrderID`.

#### **Key Clauses and Constraints**:

- `IDENTITY` column for `OrderID` and `OrderDetailID` in each table respectively, to auto-generate unique IDs for each entry.
- `DEFAULT` constraint for `OrderDate` in `dbo.Orders`, to automatically assign the current date if not specified.
- `Foreign Key` constraint linking `OrderID` in `dbo.OrderDetails` to `OrderID` in `dbo.Orders` for referential integrity.

In [2]:
USE WideWorldImporters;
GO
-- Drop tables if they already exist
DROP TABLE IF EXISTS dbo.OrderDetails;
DROP TABLE IF EXISTS dbo.Orders;

-- Create the Orders table
CREATE TABLE dbo.Orders (
    OrderID INT IDENTITY(1,1) PRIMARY KEY,
    CustomerID INT NOT NULL,
    OrderDate DATE DEFAULT GETDATE(),
    ShippedDate DATE,
    Freight DECIMAL(10, 2) NOT NULL DEFAULT 0.00,
    ShipName NVARCHAR(40),
    ShipAddress NVARCHAR(60),
    ShipCity NVARCHAR(15),
    ShipRegion NVARCHAR(15),
    ShipPostalCode NVARCHAR(10),
    ShipCountry NVARCHAR(15)
);

-- Create the OrderDetails table
CREATE TABLE dbo.OrderDetails (
    OrderDetailID INT IDENTITY(1,1) PRIMARY KEY,
    OrderID INT NOT NULL,
    ProductID INT NOT NULL,
    Quantity INT NOT NULL DEFAULT 1,
    UnitPrice DECIMAL(10, 2) NOT NULL DEFAULT 0.00,
    Discount DECIMAL(4, 3) NOT NULL DEFAULT 0.00,
    CONSTRAINT FK_OrderDetails_OrderID FOREIGN KEY (OrderID) REFERENCES dbo.Orders(OrderID)
);

-- Populate the Orders table with initial data
INSERT INTO dbo.Orders (CustomerID, OrderDate, ShippedDate, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry)
VALUES 
    (1, '2023-10-20', '2023-10-22', 25.50, 'John Doe', '123 Main St', 'New York', 'NY', '10001', 'USA'),
    (2, '2023-10-21', '2023-10-23', 10.00, 'Jane Smith', '456 Elm St', 'San Francisco', 'CA', '94101', 'USA'),
    (3, '2023-10-22', NULL, 15.00, 'Alice Brown', '789 Oak St', 'Seattle', 'WA', '98101', 'USA');

-- Populate the OrderDetails table with initial data
INSERT INTO dbo.OrderDetails (OrderID, ProductID, Quantity, UnitPrice, Discount)
VALUES 
    (1, 101, 2, 12.99, 0.05),
    (1, 102, 1, 19.99, 0.00),
    (2, 103, 3, 5.99, 0.10),
    (2, 104, 2, 9.99, 0.00),
    (3, 105, 1, 29.99, 0.00),
    (3, 106, 5, 7.99, 0.05);


Commands completed successfully.

(3 rows affected)

(6 rows affected)

Total execution time: 00:00:00.073

### **Propositions for Chapter 8 with Queries**

### Proposition 1: Creating the `ProductCategories` Table with Sample Data

**Overview:**  
This proposition creates a `ProductCategories` table with a unique, auto-generated `CategoryID` for each category and populates it with sample categories. This setup provides a foundational table to categorize products by various types.

**Key Clauses and Modifications:**
- **IDENTITY**: The `CategoryID` column is set to auto-increment, ensuring each entry has a unique identifier.
- **PRIMARY KEY**: Declares `CategoryID` as the unique key, preventing duplicate values.
- **INSERT INTO**: Adds predefined category values to the table, creating a clear structure for future entries.

In [69]:
USE WideWorldImporters;
GO

-- Step 1: Drop the ProductCategories table if it exists
DROP TABLE IF EXISTS ProductCategories;

-- Step 2: Create the ProductCategories table
CREATE TABLE ProductCategories (
    CategoryID INT IDENTITY(1,1) PRIMARY KEY,
    CategoryName NVARCHAR(50) NOT NULL
);

-- Step 3: Insert sample data into ProductCategories
INSERT INTO ProductCategories (CategoryName)
VALUES ('Electronics'), ('Clothing'), ('Furniture');

-- Verification: Retrieve data to ensure it’s added as expected
SELECT * FROM ProductCategories;


Commands completed successfully.

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.041

CategoryID,CategoryName
1,Electronics
2,Clothing
3,Furniture


### Proposition 2: Using `SCOPE_IDENTITY()` to Add Related Records

**Overview:**  
This task demonstrates inserting data into two related tables, `Orders` and `OrderDetails`. After adding an order to `Orders`, the `SCOPE_IDENTITY()` function retrieves the last generated `OrderID`, linking it to a new entry in `OrderDetails`. This setup ensures that each order and its details are consistently related.

**Key Clauses and Modifications:**
- **SCOPE_IDENTITY()**: Captures the last inserted identity value, ensuring data consistency between `Orders` and `OrderDetails`.
- **INSERT INTO**: Adds rows to both tables, linking them through the `OrderID`.

In [73]:
USE WideWorldImporters;
GO

-- Step 1: Insert a new order in the Orders table
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
VALUES (3, 20.00, 'Alice Brown', 'Seattle', 'USA');

-- Step 2: Retrieve the last inserted OrderID
DECLARE @LastOrderID INT = SCOPE_IDENTITY();

-- Step 3: Insert a new entry in OrderDetails for the retrieved OrderID
INSERT INTO dbo.OrderDetails (OrderID, ProductID, Quantity, UnitPrice)
VALUES (@LastOrderID, 104, 2, 9.99);

-- Verification: Retrieve the entry from OrderDetails to confirm
SELECT * FROM dbo.OrderDetails WHERE OrderID = @LastOrderID;


Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.036

OrderDetailID,OrderID,ProductID,Quantity,UnitPrice,Discount,ReorderLevel
16,29,104,2,9.99,0.000,10


### Proposition 3: Creating the `Invoices` Table with `InvoiceSequence`

**Overview:**  
This proposition creates a `SEQUENCE` object to generate unique `InvoiceNumber` values for `Invoices`, an approach that ensures a consistent, unique identifier for each invoice. The table also includes `InvoiceDate`, which defaults to the current date.

**Key Clauses and Modifications:**
- **SEQUENCE**: `InvoiceSequence` is used for generating unique invoice numbers, especially helpful for ordered or custom ID formats.
- **IDENTITY**: Sets `InvoiceID` to auto-increment, distinguishing it from `InvoiceNumber`.
- **DEFAULT**: Automatically assigns `InvoiceDate` as the current date if no other value is provided.Proposition 3: Create a `SEQUENCE` for generating unique invoice numbers in `Invoices`.

In [74]:
USE WideWorldImporters;
GO

-- Step 1: Drop the Invoices table if it exists, then drop the sequence
DROP TABLE IF EXISTS dbo.Invoices;
DROP SEQUENCE IF EXISTS InvoiceSequence;

-- Step 2: Create the InvoiceSequence
CREATE SEQUENCE InvoiceSequence
    START WITH 1000
    INCREMENT BY 1;

-- Step 3: Create the Invoices table with InvoiceID as an IDENTITY column
CREATE TABLE dbo.Invoices (
    InvoiceID INT IDENTITY(1,1) PRIMARY KEY,
    InvoiceNumber INT DEFAULT NEXT VALUE FOR InvoiceSequence,
    CustomerID INT,
    InvoiceDate DATE DEFAULT GETDATE(),
    TotalAmount DECIMAL(10, 2)
);

-- Step 4: Insert an invoice without specifying InvoiceID (auto-generated)
INSERT INTO dbo.Invoices (CustomerID, TotalAmount)
VALUES (1, 150.00);

-- Verification: Select to confirm InvoiceID and InvoiceNumber values
SELECT * FROM dbo.Invoices;


Commands completed successfully.

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.099

InvoiceID,InvoiceNumber,CustomerID,InvoiceDate,TotalAmount
1,1000,1,2024-10-29,150.00


### Proposition 4: Capturing Inserted Data Using the `OUTPUT` Clause

**Overview:**  
This proposition captures information about newly inserted rows in `Orders` using the `OUTPUT` clause. The captured data is temporarily stored for easy verification, allowing immediate review of inserted values.

**Key Clauses and Modifications:**
- **OUTPUT**: Directly captures and returns details of each new row, supporting transparency and data verification.
- **#Temporary Table**: The use of a temporary table for `OUTPUT` data allows data from the current session to be stored and displayed without permanently saving it in the database.

In [75]:
USE WideWorldImporters;
GO

-- Step 1: Create a temporary table to capture output
CREATE TABLE #InsertedOrders (
    OrderID INT,
    CustomerID INT,
    OrderDate DATE,
    Freight DECIMAL(10, 2)
);

-- Step 2: Insert a new order, capturing the output in #InsertedOrders
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
OUTPUT INSERTED.OrderID, INSERTED.CustomerID, INSERTED.OrderDate, INSERTED.Freight
INTO #InsertedOrders
VALUES (6, 20.00, 'Pam Beesly', 'Philadelphia', 'USA');

-- Verification: Display the captured output data
SELECT * FROM #InsertedOrders;

-- Cleanup
DROP TABLE #InsertedOrders;


Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.044

OrderID,CustomerID,OrderDate,Freight
30,6,NULL,20.00


### **Proposition 5**: Track Changes in `dbo.Orders` Using an Audit Table

In this proposition, we create a separate **audit table** called `OrderAuditLog` to track any changes in `dbo.Orders`. Whenever a new order is inserted or an existing order is updated, an entry will be added to `OrderAuditLog` to record the action.

---

#### Steps and Query:

1. **Create `OrderAuditLog` Table**: This table will store audit entries for each action performed on `dbo.Orders`, including order ID, action type (e.g., INSERT, UPDATE), and timestamp.
2. **Insert a New Order** in `dbo.Orders`, with a trigger capturing this insertion in `OrderAuditLog`.
3. **Update an Order** in `dbo.Orders`, again with the trigger capturing this change.

### Explanation

- **`OrderAuditLog` Table**: Stores audit records with `OrderID`, action type (e.g., INSERT, UPDATE), and timestamp.
- **Trigger `trg_OrderAudit`**: Automatically captures each `INSERT` or `UPDATE` on `dbo.Orders` and logs it in `OrderAuditLog`.
- **Final Verification**: The `SELECT` query on `OrderAuditLog` shows recorded actions for the tested `INSERT` and `UPDATE`.

In [76]:
USE WideWorldImporters;
GO

-- Step 1: Create the OrderAuditLog table
DROP TABLE IF EXISTS dbo.OrderAuditLog;
CREATE TABLE dbo.OrderAuditLog (
    AuditID INT IDENTITY(1,1) PRIMARY KEY,
    OrderID INT,
    Action NVARCHAR(50),
    ActionDate DATETIME DEFAULT GETDATE()
);

-- Step 2: Drop and create the trigger in a new batch
DROP TRIGGER IF EXISTS trg_OrderAudit;
GO

CREATE TRIGGER trg_OrderAudit
ON dbo.Orders
AFTER INSERT, UPDATE
AS
BEGIN
    INSERT INTO dbo.OrderAuditLog (OrderID, Action, ActionDate)
    SELECT i.OrderID, 
           CASE WHEN i.OrderID IS NOT NULL AND d.OrderID IS NULL THEN 'INSERT'
                WHEN i.OrderID IS NOT NULL AND d.OrderID IS NOT NULL THEN 'UPDATE'
           END AS Action,
           GETDATE()
    FROM inserted i
    LEFT JOIN deleted d ON i.OrderID = d.OrderID;
END;
GO

-- Step 3: Insert and Update to test the trigger
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
VALUES (7, 22.00, 'Jim Halpert', 'Scranton', 'USA');

UPDATE dbo.Orders
SET Freight = 25.00
WHERE CustomerID = 7;

-- Verification: View audit log to verify actions
SELECT * FROM dbo.OrderAuditLog;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

(6 rows affected)

(6 rows affected)

(7 rows affected)

Total execution time: 00:00:00.164

AuditID,OrderID,Action,ActionDate
1,31,INSERT,2024-10-29 03:27:20.547
2,31,UPDATE,2024-10-29 03:27:20.590
3,12,UPDATE,2024-10-29 03:27:20.590
4,11,UPDATE,2024-10-29 03:27:20.590
5,10,UPDATE,2024-10-29 03:27:20.590
6,9,UPDATE,2024-10-29 03:27:20.590
7,8,UPDATE,2024-10-29 03:27:20.590


### Proposition 6: Adding `ReorderLevel` with Default Value to `dbo.OrderDetails`

**Overview:**  
This proposition adds a `ReorderLevel` column to the `OrderDetails` table, with a default value of `10` to help manage and track stock levels automatically. The default value ensures that, unless otherwise specified, each new item added to the `OrderDetails` table will have a reorder level set.

**Steps & Explanation:**
1. **Drop Existing Constraints** – Checks for any default constraints on `ReorderLevel` and removes them if found.
2. **Drop Column if Exists** – Ensures the column `ReorderLevel` does not already exist in the table, preventing duplication.
3. **Add Column with Default** – Adds the `ReorderLevel` column with a default value of `10`.
4. **Verification** – Inserts a new row to confirm that the `ReorderLevel` field defaults to `10` if no specific value is provided.

**Key Clauses:**
- **ALTER TABLE**: Used to modify the structure of an existing table by adding or removing columns.
- **DEFAULT**: Specifies a default value for new rows added to the table.



In [111]:
USE WideWorldImporters;
GO

-- Step 1: Identify and drop the default constraint on ReorderLevel, if it exists
DECLARE @constraintName NVARCHAR(200);

SELECT @constraintName = name 
FROM sys.default_constraints 
WHERE parent_object_id = OBJECT_ID('dbo.OrderDetails') 
  AND parent_column_id = COLUMNPROPERTY(OBJECT_ID('dbo.OrderDetails'), 'ReorderLevel', 'ColumnId');

IF @constraintName IS NOT NULL
BEGIN
    EXEC('ALTER TABLE dbo.OrderDetails DROP CONSTRAINT ' + @constraintName);
END

-- Step 2: Drop the ReorderLevel column if it exists
IF EXISTS (SELECT * FROM sys.columns 
           WHERE Name = N'ReorderLevel' AND Object_ID = Object_ID(N'dbo.OrderDetails'))
BEGIN
    ALTER TABLE dbo.OrderDetails
    DROP COLUMN ReorderLevel;
END;

-- Step 3: Add the ReorderLevel column with a default value of 10
ALTER TABLE dbo.OrderDetails
ADD ReorderLevel INT DEFAULT 10;

-- Test: Insert a new row to confirm ReorderLevel default value
INSERT INTO dbo.OrderDetails (OrderID, ProductID, Quantity, UnitPrice, Discount)
VALUES (1, 107, 3, 15.00, 0.00);

-- Verify that ReorderLevel has the default value of 10 for the new row
SELECT OrderID, ProductID, Quantity, UnitPrice, Discount, ReorderLevel
FROM dbo.OrderDetails
WHERE ProductID = 107;


Commands completed successfully.

(1 row affected)

(5 rows affected)

Total execution time: 00:00:00.070

OrderID,ProductID,Quantity,UnitPrice,Discount,ReorderLevel
1,107,3,15.00,0.000,NULL
1,107,3,15.00,0.000,NULL
1,107,3,15.00,0.000,NULL
1,107,3,15.00,0.000,NULL
1,107,3,15.00,0.000,10


### Proposition 7: Adding `LastUpdated` Column with `GETDATE()` Default to `dbo.Orders`

**Overview:**  
The `LastUpdated` column in `dbo.Orders` tracks the date and time of the last update for each order. This default value of `GETDATE()` records the exact moment an entry is created or modified, providing an accurate timeline of changes.

**Steps & Explanation:**
1. **Drop Existing Constraints** – Searches for and removes any existing default constraints on the `LastUpdated` column.
2. **Drop Column if Exists** – Ensures there are no duplicate `LastUpdated` columns.
3. **Add Column with Default** – Adds `LastUpdated` to `dbo.Orders` with `GETDATE()` as the default.
4. **Verification** – Inserts a row to verify that `LastUpdated` correctly defaults to the current timestamp.

**Key Clauses:**
- **GETDATE()**: Captures the current date and time, which is applied to `LastUpdated`.
- **ALTER TABLE**: Modifies `dbo.Orders` to add the `LastUpdated` column with the default.

In [112]:
USE WideWorldImporters;
GO

-- Step 1: Drop existing default constraint on LastUpdated if it exists
DECLARE @constraintName NVARCHAR(200), @sql NVARCHAR(MAX) = '';
SELECT @constraintName = name 
FROM sys.default_constraints 
WHERE parent_object_id = OBJECT_ID('dbo.Orders') 
  AND parent_column_id = COLUMNPROPERTY(OBJECT_ID('dbo.Orders'), 'LastUpdated', 'ColumnId');
IF @constraintName IS NOT NULL
BEGIN
    SET @sql = 'ALTER TABLE dbo.Orders DROP CONSTRAINT ' + QUOTENAME(@constraintName) + ';';
    EXEC sp_executesql @sql;
END;

-- Step 2: Drop the LastUpdated column if it exists
IF EXISTS (SELECT * FROM sys.columns WHERE Name = N'LastUpdated' AND Object_ID = Object_ID(N'dbo.Orders'))
BEGIN
    ALTER TABLE dbo.Orders
    DROP COLUMN LastUpdated;
END;

-- Step 3: Add the LastUpdated column with default value of GETDATE()
ALTER TABLE dbo.Orders ADD LastUpdated DATETIME DEFAULT GETDATE();

-- Verification: Insert a new order to test the LastUpdated default value
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
VALUES (13, 50.00, 'Angela Martin', 'Scranton', 'USA');

-- Verify that LastUpdated has the default value for the new row
SELECT OrderID, CustomerID, OrderDate, Freight, LastUpdated
FROM dbo.Orders
WHERE CustomerID = 13;


Commands completed successfully.

(1 row affected)

(1 row affected)

(10 rows affected)

Total execution time: 00:00:00.109

OrderID,CustomerID,OrderDate,Freight,LastUpdated
13,13,NULL,50.00,NULL
14,13,NULL,50.00,NULL
15,13,NULL,50.00,NULL
16,13,NULL,50.00,NULL
17,13,NULL,50.00,NULL
18,13,NULL,50.00,NULL
19,13,NULL,50.00,NULL
41,13,NULL,50.00,NULL
42,13,NULL,50.00,NULL
43,13,NULL,50.00,2024-10-29 03:42:28.577


### Proposition 8: Creating `ReceiptLogs` Table with `SEQUENCE` for `ReceiptNumber`

**Overview:**  
The `ReceiptLogs` table uses a `SEQUENCE` to auto-generate unique `ReceiptNumber` values, ensuring unique identifiers for each receipt. This setup demonstrates the flexibility of using sequences for ordered numbering schemes.

**Key Clauses and Modifications:**
- **SEQUENCE**: Provides an ordered number for each receipt, a simpler alternative to identity columns for managing non-primary keys.
- **IDENTITY**: Auto-generates unique IDs for each receipt entry.
- **DEFAULT**: Applies the current date to `TransactionDate`, maintaining consistency for receipts.

In [136]:
-- Step 1: Drop Existing Dependencies and the `ReceiptSeq` SEQUENCE

USE WideWorldImporters;
GO

DECLARE @constraintName NVARCHAR(200), @sql NVARCHAR(MAX);

-- Identify and drop any default constraint tied to ReceiptNumber
SELECT @constraintName = name 
FROM sys.default_constraints 
WHERE parent_object_id = OBJECT_ID('dbo.ReceiptLogs') 
  AND parent_column_id = COLUMNPROPERTY(OBJECT_ID('dbo.ReceiptLogs'), 'ReceiptNumber', 'ColumnId');

IF @constraintName IS NOT NULL
BEGIN
    SET @sql = 'ALTER TABLE dbo.ReceiptLogs DROP CONSTRAINT ' + QUOTENAME(@constraintName) + ';';
    EXEC sp_executesql @sql;
END;

-- Drop the sequence if it exists
DROP SEQUENCE IF EXISTS ReceiptSeq;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.032

In [131]:
-- Step 2: Recreate the `ReceiptSeq` SEQUENCE and `ReceiptLogs` Table
--This step creates a fresh sequence and the `ReceiptLogs` table using that sequence.
USE WideWorldImporters;
GO

-- Create the sequence
CREATE SEQUENCE ReceiptSeq
    START WITH 1000
    INCREMENT BY 1;
GO

-- Create the ReceiptLogs table with ReceiptNumber linked to ReceiptSeq
DROP TABLE IF EXISTS ReceiptLogs;
CREATE TABLE ReceiptLogs (
    ReceiptID INT PRIMARY KEY IDENTITY(1,1),
    ReceiptNumber INT DEFAULT NEXT VALUE FOR ReceiptSeq,
    TransactionDate DATE DEFAULT GETDATE(),
    Amount DECIMAL(10, 2) NOT NULL
);
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.047

In [137]:
-- Step 3: Insert Sample Data and Verify
--- This step inserts sample data to confirm the functionality of `ReceiptNumber` generation.

USE WideWorldImporters;
GO

-- Insert sample data to check ReceiptNumber generation
INSERT INTO ReceiptLogs (Amount)
VALUES (150.00), (200.00);

-- Verification: Check the ReceiptLogs table
SELECT * FROM ReceiptLogs;
GO


Commands completed successfully.

(2 rows affected)

(4 rows affected)

Total execution time: 00:00:00.018

ReceiptID,ReceiptNumber,TransactionDate,Amount
1,1000,2024-10-29,150.00
2,1001,2024-10-29,200.00
3,NULL,2024-10-29,150.00
4,NULL,2024-10-29,200.00


### Proposition 9: Adding an `IDENTITY` Column `SerialNumber` to `dbo.Inventory`

**Overview:**  
Adds a `SerialNumber` column to `Inventory`, using `IDENTITY` to uniquely identify each inventory item. This setup supports straightforward tracking of each item without requiring an additional key.

**Key Clauses and Modifications:**
- **IDENTITY**: Generates a unique `SerialNumber` for each row in `Inventory`, simplifying tracking.
- **ALTER TABLE**: Modifies `Inventory` by adding the new `SerialNumber` column.

In [143]:
USE WideWorldImporters
GO

USE WideWorldImporters;
GO

DECLARE @constraintName NVARCHAR(200), @sql NVARCHAR(MAX) = '';

-- Find and drop the default constraint on IsPriority if it exists
SELECT @constraintName = name 
FROM sys.default_constraints 
WHERE parent_object_id = OBJECT_ID('dbo.Orders') 
  AND parent_column_id = COLUMNPROPERTY(OBJECT_ID('dbo.Orders'), 'IsPriority', 'ColumnId');

IF @constraintName IS NOT NULL
BEGIN
    SET @sql = 'ALTER TABLE dbo.Orders DROP CONSTRAINT ' + QUOTENAME(@constraintName) + ';';
    EXEC sp_executesql @sql;
END;


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.006

In [144]:
USE WideWorldImporters
GO

IF EXISTS (SELECT * FROM sys.columns 
           WHERE Name = N'IsPriority' AND Object_ID = Object_ID(N'dbo.Orders'))
BEGIN
    ALTER TABLE dbo.Orders
    DROP COLUMN IsPriority;
END;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.043

In [145]:
USE WideWorldImporters
GO

ALTER TABLE dbo.Orders
ADD IsPriority BIT DEFAULT 0;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.012

In [146]:
USE WideWorldImporters
GO

-- Insert a new order to test the IsPriority default value
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
VALUES (14, 60.00, 'Creed Bratton', 'Akron', 'USA');

-- Verify that IsPriority has the default value for the new row
SELECT OrderID, CustomerID, OrderDate, Freight, IsPriority
FROM dbo.Orders
WHERE CustomerID = 14;


Commands completed successfully.

(1 row affected)

(1 row affected)

(8 rows affected)

Total execution time: 00:00:00.022

OrderID,CustomerID,OrderDate,Freight,IsPriority
20,14,NULL,60.00,NULL
21,14,NULL,60.00,NULL
22,14,NULL,60.00,NULL
23,14,NULL,60.00,NULL
35,14,NULL,60.00,NULL
38,14,NULL,60.00,NULL
45,14,NULL,60.00,NULL
46,14,NULL,60.00,0


### Proposition 10: Adding an `OrderNote` Column to `dbo.Orders` with Default Value `'No Notes'`

**Overview:**  
The `OrderNote` column is added to `Orders`, allowing optional notes for each order with a default value of `'No Notes'` for entries lacking specific notes.

**Key Clauses and Modifications:**
- **ALTER TABLE**: Adds a new column, `OrderNote`, to the table.
- **DEFAULT**: Applies `'No Notes'` to `OrderNote` by default, ensuring clarity for orders with no additional comments.


In [87]:
-- Step 1: Drop Any Existing Constraints on `OrderNote`

USE WideWorldImporters;
GO

DECLARE @constraintName NVARCHAR(200), @sql NVARCHAR(MAX) = '';

-- Find and drop the default constraint on OrderNote if it exists
SELECT @constraintName = name 
FROM sys.default_constraints 
WHERE parent_object_id = OBJECT_ID('dbo.Orders') 
  AND parent_column_id = COLUMNPROPERTY(OBJECT_ID('dbo.Orders'), 'OrderNote', 'ColumnId');

IF @constraintName IS NOT NULL
BEGIN
    SET @sql = 'ALTER TABLE dbo.Orders DROP CONSTRAINT ' + QUOTENAME(@constraintName) + ';';
    EXEC sp_executesql @sql;
END;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.043

In [88]:
--- Step 2: Drop the `OrderNote` Column if it Exists


IF EXISTS (SELECT * FROM sys.columns 
           WHERE Name = N'OrderNote' AND Object_ID = Object_ID(N'dbo.Orders'))
BEGIN
    ALTER TABLE dbo.Orders
    DROP COLUMN OrderNote;
END;



Commands completed successfully.

Total execution time: 00:00:00.044

In [89]:
--- Step 3: Add the `OrderNote` Column with a Default Value of `'No Notes'`

ALTER TABLE dbo.Orders
ADD OrderNote NVARCHAR(100) DEFAULT 'No Notes';



Commands completed successfully.

Total execution time: 00:00:00.010

In [90]:
---- Step 4: Insert a Test Row to Confirm the Default Value

-- Insert a new order to test the OrderNote default value
INSERT INTO dbo.Orders (CustomerID, Freight, ShipName, ShipCity, ShipCountry)
VALUES (16, 35.00, 'Toby Flenderson', 'Scranton', 'USA');

-- Verify that OrderNote has the default value for the new row
SELECT OrderID, CustomerID, OrderDate, Freight, OrderNote
FROM dbo.Orders
WHERE CustomerID = 16;



(1 row affected)

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.045

OrderID,CustomerID,OrderDate,Freight,OrderNote
25,16,NULL,35.00,NULL
36,16,NULL,35.00,NULL
37,16,NULL,35.00,No Notes
